In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 13 14:21:08 2022

@author: jamesonblount
"""

# In[]:
# Importing required packages
#Importing basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Importing sklearn modules
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve
from sklearn import ensemble, linear_model, neighbors, svm, tree, neural_network
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn import svm,model_selection, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


from sklearn.ensemble import RandomForestClassifier




In [2]:
# In[]:
#Loading the data and checking for missing values
dataset=pd.read_csv('C:/Users/ictinike/Documents/WrayLab/raw_data/x_0011_df_phyloP.csv')
dataset.isnull().sum()

datasetv2 = dataset.dropna(axis=1)
datasetv2.isnull().sum()

seqnames                  0
start                     0
end                       0
width                     0
strand                    0
                         ..
gene.y                    0
dTSS                      0
PhastCons                 0
PhyloP_primates_score     0
PhyloP_placental_score    0
Length: 90, dtype: int64

In [3]:
dataset.head()

,seqnames,start,end,width,strand,name,score,signalValue,pValue,qValue,...,S_rare,eQTLs,PP_con,PP_acc,gene.y,dTSS,PhastCons,PhyloP_mammals_score,PhyloP_primates_score,PhyloP_placental_score
0,chr1,793301,793692,392,*,chr1.7,653,0.0677,3.87,-1,...,5,0,0.951590,0.293804,SAMD11,-67621,0.003,-0.164622,-0.078358,-0.103762
1,chr1,793779,794382,604,*,chr1.8,553,0.0359,1.98,-1,...,11,4,0.948498,0.189047,SAMD11,-67037,0.001,-0.060388,-0.150270,-0.147019
2,chr1,846424,847133,710,*,chr1.17,867,0.1357,7.97,-1,...,10,5,0.927576,0.266338,SAMD11,-14339,0.000,-0.080633,-0.131020,-0.100292
3,chr1,847379,847941,563,*,chr1.18,544,0.0332,1.82,-1,...,1,2,0.143830,1.000000,SAMD11,-13458,0.001,-0.172059,-0.024884,-0.058467
4,chr1,848207,849945,1739,*,chr1.19,605,0.0525,2.96,-1,...,24,11,0.342990,0.833725,SAMD11,-12042,0.000,-0.032988,-0.135259,-0.177847


In [5]:
    # Transforming the categorical variables into numerical
# Instantiate OneHotEncoder
ohe = OneHotEncoder(sparse = False)
ohe.fit_transform(datasetv2[["chromHMM_cat_longest"]])[:5]
datasetv2['chromHMM_cat_longest'].head()
ohe.categories_



[array(['Active Promoter', 'Candidate Strong Enhancer',
        'Candidate Weak Enhancer', 'Distal CTCF/Candidate Insulator',
        'Heterochromatin/Repetitive/Copy Number Variation',
        'Inactive Promoter', 'Low activity proximal to active states',
        'Polycomb repressed', 'Promoter Flanking',
        'Transcription asociated'], dtype=object)]

In [4]:
datasetv2[["chromHMM_cat_longest"]]

,chromHMM_cat_longest
0,Distal CTCF/Candidate Insulator
1,Candidate Weak Enhancer
2,Candidate Weak Enhancer
3,Candidate Weak Enhancer
4,Polycomb repressed
...,...
80348,Active Promoter
80349,Distal CTCF/Candidate Insulator
80350,Candidate Strong Enhancer
80351,Candidate Strong Enhancer


In [8]:
ohe.fit_transform(datasetv2[["chromHMM_cat_longest"]])[:5]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [6]:
# In[]:
# Compare performance of ML between top quartile to bottom quartile of AUC (best)/peak size
bin_labels = ['Lower', 'Midlower','Midupper', 'Upper']
lower_bin = ['Lower']
upper_bin = ['Upper']
datasetv2['score_quart'] = pd.qcut(datasetv2['score'], q = 4, labels = bin_labels)
datasetLower = datasetv2[datasetv2['score_quart'].isin(lower_bin)]
datasetUpper = datasetv2[datasetv2['score_quart'].isin(upper_bin)]

C:\Users\Ictinike\AppData\Local\Temp\ipykernel_14592\2677276402.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasetv2['score_quart'] = pd.qcut(datasetv2['score'], q = 4, labels = bin_labels)


In [24]:
datasetUpper.head()

,seqnames,start,end,width,strand,name,score,signalValue,pValue,qValue,...,S_rare,eQTLs,PP_con,PP_acc,gene.y,dTSS,PhastCons,PhyloP_primates_score,PhyloP_placental_score,score_quart
2,chr1,846424,847133,710,*,chr1.17,867,0.1357,7.97,-1,...,10,5,0.927576,0.266338,SAMD11,-14339,0.000,-0.131020,-0.100292,Upper
9,chr1,856118,856944,827,*,chr1.24,1000,0.3804,16.00,-1,...,10,5,0.376009,0.912880,SAMD11,-4587,0.001,-0.056528,-0.023788,Upper
12,chr1,858457,863327,4871,*,chr1.27,1000,0.2138,12.70,-1,...,50,14,0.668905,0.620494,SAMD11,-226,0.001,-0.072562,0.036639,Upper
14,chr1,870706,881162,10457,*,chr1.34,1000,0.4497,16.00,-1,...,177,38,0.238055,0.840641,SAMD11,14816,0.000,-0.079952,-0.064100,Upper
18,chr1,893900,897154,3255,*,chr1.40,1000,0.4485,16.00,-1,...,39,14,0.877861,0.309112,KLHL17,-440,0.000,-0.151414,-0.121736,Upper


In [25]:
    # here we'll start by using wgCERES_score_nosig as the response vector,
x = datasetv2[["DHS_prop_repeat", 
                    "DHS_prop_GC", "DHS_length", "n_SNV_Zhou_per_bp", 
                    "distanceToTSS", "zeta.human", "zeta.chimp", "PP_con", "PP_acc", 
                    "PhastCons",
                    "chromHMM_cat_longest", 
                    "annotation", "PhyloP_primates_score"]]
y = datasetv2["wgCERES_score_nosig"]

In [26]:
# In[]:
    # Make column transformer
column_transform = []
column_transform = make_column_transformer(
    (ohe, ['chromHMM_cat_longest','annotation']))

#Apply column transformer to predictor variables
column_transform.fit(x)


# In[]:
# Splitting train and split data
# The test data set size is 20% of the total records. This test data will not 
# be used in model training and work as an independent test data.
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=0)

In [27]:
# Building the rest of the pipeline
# Instantiate pipeline with linear regression
lm  = LinearRegression()
lm_pipeline = make_pipeline(column_transform, lm)

In [28]:

# In[]:
# Instantiate pipeline with gradient boosting
gbm = GradientBoostingRegressor()
gbm_pipeline = make_pipeline(column_transform, gbm)

# In[]:
# Instantiate pipeline with logistic regression
lr = LogisticRegression()
lr_pipeline = make_pipeline(column_transform, lr)

# In[]:
# Instantiate pipeline for SVM
sv = SVC()
sv_pipeline = make_pipeline(column_transform, sv)

In [29]:
# In[]:
# Fit pipeline to training set and make predictions on test set

lm_pipeline.fit(x_train, y_train)
lm_predictions = lm_pipeline.predict(x_test)
print("First 5 LM predictions: ", list(lm_predictions[:5]))

gbm_pipeline.fit(x_train, y_train)
gbm_predictions = gbm_pipeline.predict(x_test)
print("First 5 GBM predictions: ", list(gbm_predictions[:5]))

First 5 LM predictions:  [0.34375, 0.28125, 0.24609375, 0.34375, 0.07421875]
First 5 GBM predictions:  [0.3255569131908791, 0.17682304921118197, 0.21815800507641656, 0.3164706985918748, 0.06331650004906972]


In [30]:

# With predictions ready from the two pipelines, we can proceed to evaluate the 
# accuracy of these predictions using mean absolute error (MAE) and mean squared 
# error (RMSE).
# Calculate mean square error and root mean squared error

lm_mae = mean_absolute_error(lm_predictions, y_test)
lm_rmse = np.sqrt(mean_squared_error(lm_predictions, y_test))
print("LM MAE: {:.2f}".format(round(lm_mae, 2)))
print("LM RMSE: {:.2f}".format(round(lm_rmse, 2)))

LM MAE: 1.28
LM RMSE: 2.00


In [31]:
gbm_mae = mean_absolute_error(gbm_predictions, y_test)
gbm_rmse = np.sqrt(mean_squared_error(gbm_predictions, y_test))
print("GBM MAE: {:.2f}".format(round(gbm_mae, 2)))
print("GBM RMSE: {:.2f}".format(round(gbm_rmse, 2)))

GBM MAE: 1.29
GBM RMSE: 1.99


In [ ]:


# In[]:
    # here we'll use dhs_0_1_wg as the response vector for the classifiers
x = datasetv2[["DHS_prop_repeat", 
                    "DHS_prop_GC", "DHS_length", "n_SNV_Zhou_per_bp", 
                    "distanceToTSS", "zeta.human", "zeta.chimp", "PP_con", "PP_acc", 
                    "PhastCons",
                    "chromHMM_cat_longest", 
                    "annotation", "PhyloP_primates_score"]]
y = datasetv2["dhs_0_1_wg"]

# In[]:
    # Make column transformer
column_transform = []
column_transform = make_column_transformer(
    (ohe, ['chromHMM_cat_longest','annotation']))

#Apply column transformer to predictor variables
column_transform.fit(x)

In [ ]:
# In[]:
# Splitting train and split data
# The test data set size is 20% of the total records. This test data will not 
# be used in model training and work as an independent test data.
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=0)
# In[]:

lr_pipeline.fit(x_train, y_train)
lr_predictions = lr_pipeline.predict(x_test)
print("First 5 LR predictions: ", list(lr_predictions[:5]))

sv_pipeline.fit(x_train, y_train)
sv_predictions = sv_pipeline.predict(x_test)
print("First 5 SVM predictions: ", list(sv_predictions[:5]))

In [ ]:
# With predictions ready from the two pipelines, we can proceed to evaluate the 
# accuracy of these predictions using mean absolute error (MAE) and mean squared 
# error (RMSE).
# Calculate mean square error and root mean squared error

lr_mae = mean_absolute_error(lr_predictions, y_test)
lr_rmse = np.sqrt(mean_squared_error(lr_predictions, y_test))
print("LR MAE: {:.2f}".format(round(lr_mae, 2)))
print("LR RMSE: {:.2f}".format(round(lr_rmse, 2)))

sv_mae = mean_absolute_error(sv_predictions, y_test)
sv_rmse = np.sqrt(mean_squared_error(sv_predictions, y_test))
print("SVM MAE: {:.2f}".format(round(sv_mae, 2)))
print("SVM RMSE: {:.2f}".format(round(sv_rmse, 2)))

In [ ]:
# Performing this same pipeline but using the extreme subsets of "score"
# In[]:
    # here we'll start by using wgCERES_score_nosig as the response vector,
x = datasetLower[["DHS_prop_repeat", 
                    "DHS_prop_GC", "DHS_length", "n_SNV_Zhou_per_bp", 
                    "distanceToTSS", "zeta.human", "zeta.chimp", "PP_con", "PP_acc", 
                    "PhastCons",
                    "chromHMM_cat_longest", 
                    "annotation", "PhyloP_primates_score"]]
y = datasetLower["wgCERES_score_nosig"]
# For classifier ML techniques, i.e. SVM and Random Forest
# y = datasetLower["dhs_0_1_wg"]

# In[]:
    # Make column transformer
column_transform = []
column_transform = make_column_transformer(
    (ohe, ['chromHMM_cat_longest','annotation']))

#Apply column transformer to predictor variables
column_transform.fit(x)

In [ ]:
# In[]:
# Splitting train and split data
# The test data set size is 20% of the total records. This test data will not 
# be used in model training and work as an independent test data.
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=0)

# In[]:
# Building the rest of the pipeline
# Instantiate pipeline with linear regression
lm  = LinearRegression()
lm_pipeline = make_pipeline(column_transform, lm)

# In[]:
# Instantiate pipeline with gradient boosting
gbm = GradientBoostingRegressor()
gbm_pipeline = make_pipeline(column_transform, gbm)

# In[]:
# Instantiate pipeline with logistic regression
lr = LogisticRegression()
lr_pipeline = make_pipeline(column_transform, lr)

# In[]:
# Instantiate pipeline for SVM
sv = SVC()
sv_pipeline = make_pipeline(column_transform, sv)


In [ ]:

# In[]:
# Fit pipeline to training set and make predictions on test set

lm_pipeline.fit(x_train, y_train)
lm_predictions = lm_pipeline.predict(x_test)
print("First 5 LM predictions: ", list(lm_predictions[:5]))

gbm_pipeline.fit(x_train, y_train)
gbm_predictions = gbm_pipeline.predict(x_test)
print("First 5 GBM predictions: ", list(gbm_predictions[:5]))


# In[]:
# With predictions ready from the two pipelines, we can proceed to evaluate the 
# accuracy of these predictions using mean absolute error (MAE) and mean squared 
# error (RMSE).
# Calculate mean square error and root mean squared error

lm_mae = mean_absolute_error(lm_predictions, y_test)
lm_rmse = np.sqrt(mean_squared_error(lm_predictions, y_test))
print("LM MAE: {:.2f}".format(round(lm_mae, 2)))
print("LM RMSE: {:.2f}".format(round(lm_rmse, 2)))

gbm_mae = mean_absolute_error(gbm_predictions, y_test)
gbm_rmse = np.sqrt(mean_squared_error(gbm_predictions, y_test))
print("GBM MAE: {:.2f}".format(round(gbm_mae, 2)))
print("GBM RMSE: {:.2f}".format(round(gbm_rmse, 2)))

In [ ]:
# In[]:
    # here we'll use dhs_0_1_wg as the response vector for the classifiers
x = datasetLower[["DHS_prop_repeat", 
                    "DHS_prop_GC", "DHS_length", "n_SNV_Zhou_per_bp", 
                    "distanceToTSS", "zeta.human", "zeta.chimp", "PP_con", "PP_acc", 
                    "PhastCons",
                    "chromHMM_cat_longest", 
                    "annotation", "PhyloP_primates_score"]]
y = datasetLower["dhs_0_1_wg"]

# In[]:
    # Make column transformer
column_transform = []
column_transform = make_column_transformer(
    (ohe, ['chromHMM_cat_longest','annotation']))

#Apply column transformer to predictor variables
column_transform.fit(x)

In [ ]:
# In[]:
# Splitting train and split data
# The test data set size is 20% of the total records. This test data will not 
# be used in model training and work as an independent test data.
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=0)
# In[]:

lr_pipeline.fit(x_train, y_train)
lr_predictions = lr_pipeline.predict(x_test)
print("First 5 LR predictions: ", list(lr_predictions[:5]))

sv_pipeline.fit(x_train, y_train)
sv_predictions = sv_pipeline.predict(x_test)
print("First 5 SVM predictions: ", list(sv_predictions[:5]))

#rf_pipeline.fit(x_train, y_train)
#rf_predictions = rf_pipeline.predict(x_test)
#print("First 5 RF predictions: ", list(rf_predictions[:5]))
# To use random forest, need binary outcome

In [ ]:
# In[]:
# With predictions ready from the two pipelines, we can proceed to evaluate the 
# accuracy of these predictions using mean absolute error (MAE) and mean squared 
# error (RMSE).
# Calculate mean square error and root mean squared error

lr_mae = mean_absolute_error(lr_predictions, y_test)
lr_rmse = np.sqrt(mean_squared_error(lr_predictions, y_test))
print("LR MAE: {:.2f}".format(round(lr_mae, 2)))
print("LR RMSE: {:.2f}".format(round(lr_rmse, 2)))

sv_mae = mean_absolute_error(sv_predictions, y_test)
sv_rmse = np.sqrt(mean_squared_error(sv_predictions, y_test))
print("SVM MAE: {:.2f}".format(round(sv_mae, 2)))
print("SVM RMSE: {:.2f}".format(round(sv_rmse, 2)))


In [ ]:
# In[]:
    # here we'll start by using wgCERES_score_nosig as the response vector,
x = datasetUpper[["DHS_prop_repeat", 
                    "DHS_prop_GC", "DHS_length", "n_SNV_Zhou_per_bp", 
                    "distanceToTSS", "zeta.human", "zeta.chimp", "PP_con", "PP_acc", 
                    "PhastCons",
                    "chromHMM_cat_longest", 
                    "annotation", "PhyloP_primates_score"]]
y = datasetUpper["wgCERES_score_nosig"]
# For classifier ML techniques, i.e. SVM and Random Forest
# y = datasetUpper["dhs_0_1_wg"]

# In[]:
    # Make column transformer
column_transform = []
column_transform = make_column_transformer(
    (ohe, ['chromHMM_cat_longest','annotation']))

In [ ]:
#Apply column transformer to predictor variables
column_transform.fit(x)


# In[]:
# Splitting train and split data
# The test data set size is 20% of the total records. This test data will not 
# be used in model training and work as an independent test data.
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=0)

In [ ]:
# In[]:
# Building the rest of the pipeline
# Instantiate pipeline with linear regression
lm  = LinearRegression()
lm_pipeline = make_pipeline(column_transform, lm)

# In[]:
# Instantiate pipeline with gradient boosting
gbm = GradientBoostingRegressor()
gbm_pipeline = make_pipeline(column_transform, gbm)

# In[]:
# Instantiate pipeline with logistic regression
lr = LogisticRegression()
lr_pipeline = make_pipeline(column_transform, lr)

In [ ]:
# In[]:
# Instantiate pipeline for SVM
sv = SVC()
sv_pipeline = make_pipeline(column_transform, sv)

# In[]:
# Fit pipeline to training set and make predictions on test set

lm_pipeline.fit(x_train, y_train)
lm_predictions = lm_pipeline.predict(x_test)
print("First 5 LM predictions: ", list(lm_predictions[:5]))

gbm_pipeline.fit(x_train, y_train)
gbm_predictions = gbm_pipeline.predict(x_test)
print("First 5 GBM predictions: ", list(gbm_predictions[:5]))

In [ ]:

# In[]:
# With predictions ready from the two pipelines, we can proceed to evaluate the 
# accuracy of these predictions using mean absolute error (MAE) and mean squared 
# error (RMSE).
# Calculate mean square error and root mean squared error

lm_mae = mean_absolute_error(lm_predictions, y_test)
lm_rmse = np.sqrt(mean_squared_error(lm_predictions, y_test))
print("LM MAE: {:.2f}".format(round(lm_mae, 2)))
print("LM RMSE: {:.2f}".format(round(lm_rmse, 2)))

gbm_mae = mean_absolute_error(gbm_predictions, y_test)
gbm_rmse = np.sqrt(mean_squared_error(gbm_predictions, y_test))
print("GBM MAE: {:.2f}".format(round(gbm_mae, 2)))
print("GBM RMSE: {:.2f}".format(round(gbm_rmse, 2)))


In [ ]:
# In[]:
    # here we'll use dhs_0_1_wg as the response vector for the classifiers
x = datasetUpper[["DHS_prop_repeat", 
                    "DHS_prop_GC", "DHS_length", "n_SNV_Zhou_per_bp", 
                    "distanceToTSS", "zeta.human", "zeta.chimp", "PP_con", "PP_acc", 
                    "PhastCons",
                    "chromHMM_cat_longest", 
                    "annotation", "PhyloP_primates_score"]]
y = datasetUpper["dhs_0_1_wg"]

# In[]:
    # Make column transformer
column_transform = []
column_transform = make_column_transformer(
    (ohe, ['chromHMM_cat_longest','annotation']))

#Apply column transformer to predictor variables
column_transform.fit(x)


In [ ]:
# In[]:
# Splitting train and split data
# The test data set size is 20% of the total records. This test data will not 
# be used in model training and work as an independent test data.
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=0)
# In[]:

lr_pipeline.fit(x_train, y_train)
lr_predictions = lr_pipeline.predict(x_test)
print("First 5 LR predictions: ", list(lr_predictions[:5]))

sv_pipeline.fit(x_train, y_train)
sv_predictions = sv_pipeline.predict(x_test)
print("First 5 SVM predictions: ", list(sv_predictions[:5]))

#rf_pipeline.fit(x_train, y_train)
#rf_predictions = rf_pipeline.predict(x_test)
#print("First 5 RF predictions: ", list(rf_predictions[:5]))
# To use random forest, need binary outcome

In [ ]:
# In[]:
# With predictions ready from the two pipelines, we can proceed to evaluate the 
# accuracy of these predictions using mean absolute error (MAE) and mean squared 
# error (RMSE).
# Calculate mean square error and root mean squared error

lr_mae = mean_absolute_error(lr_predictions, y_test)
lr_rmse = np.sqrt(mean_squared_error(lr_predictions, y_test))
print("LR MAE: {:.2f}".format(round(lr_mae, 2)))
print("LR RMSE: {:.2f}".format(round(lr_rmse, 2)))

sv_mae = mean_absolute_error(sv_predictions, y_test)
sv_rmse = np.sqrt(mean_squared_error(sv_predictions, y_test))
print("SVM MAE: {:.2f}".format(round(sv_mae, 2)))
print("SVM RMSE: {:.2f}".format(round(sv_rmse, 2)))

In [ ]:
# In[]:
    # After filtering for OCRs that are only present within certain TADs,
    # reading in that dataset here and testing the harness again
dataset=pd.read_csv('/Users/jamesonblount/Documents/Wray_Rotation/Carl/X_0012/data/OCRs_inTADs.csv')
dataset.isnull().sum()

datasetv2 = dataset.dropna(axis=1)
datasetv2.isnull().sum()
# Checking the data set for any NULL values is very essential, as MLAs can not 
# handle NULL values. We have to either eliminate the records with NULL values 
# or replace them with the mean/median of the other values. we can see each of 
# the variables are printed with number of null values. This data set has no null 
# values so all are zero here.
# In[]:
    # here we'll start by using wgCERES_score_nosig as the response vector,
x = datasetv2[["DHS_prop_repeat", 
                    "DHS_prop_GC", "DHS_length", "n_SNV_Zhou_per_bp", 
                    "distanceToTSS", "zeta.human", "zeta.chimp", "PP_con", "PP_acc", 
                    "PhastCons",
                    "chromHMM_cat_longest", 
                    "annotation", "PhyloP_primates_score"]]
y = datasetv2["wgCERES_score_nosig"]
# For classifier ML techniques, i.e. SVM and Random Forest
# y = datasetv2["dhs_0_1_wg"]

# In[]:
    # Make column transformer
column_transform = []
column_transform = make_column_transformer(
    (ohe, ['chromHMM_cat_longest','annotation']))

#Apply column transformer to predictor variables
column_transform.fit(x)


# In[]:
# Splitting train and split data
# The test data set size is 20% of the total records. This test data will not 
# be used in model training and work as an independent test data.
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=0)

# In[]:
# Building the rest of the pipeline
# Instantiate pipeline with linear regression
lm  = LinearRegression()
lm_pipeline = make_pipeline(column_transform, lm)

# In[]:
# Instantiate pipeline with gradient boosting
gbm = GradientBoostingRegressor()
gbm_pipeline = make_pipeline(column_transform, gbm)

# In[]:
# Instantiate pipeline with logistic regression
lr = LogisticRegression()
lr_pipeline = make_pipeline(column_transform, lr)

# In[]:
# Instantiate pipeline for SVM
sv = SVC()
sv_pipeline = make_pipeline(column_transform, sv)

# In[]:
# Fit pipeline to training set and make predictions on test set

lm_pipeline.fit(x_train, y_train)
lm_predictions = lm_pipeline.predict(x_test)
print("First 5 LM predictions: ", list(lm_predictions[:5]))

gbm_pipeline.fit(x_train, y_train)
gbm_predictions = gbm_pipeline.predict(x_test)
print("First 5 GBM predictions: ", list(gbm_predictions[:5]))


# In[]:
# With predictions ready from the two pipelines, we can proceed to evaluate the 
# accuracy of these predictions using mean absolute error (MAE) and mean squared 
# error (RMSE).
# Calculate mean square error and root mean squared error

lm_mae = mean_absolute_error(lm_predictions, y_test)
lm_rmse = np.sqrt(mean_squared_error(lm_predictions, y_test))
print("LM MAE: {:.2f}".format(round(lm_mae, 2)))
print("LM RMSE: {:.2f}".format(round(lm_rmse, 2)))

gbm_mae = mean_absolute_error(gbm_predictions, y_test)
gbm_rmse = np.sqrt(mean_squared_error(gbm_predictions, y_test))
print("GBM MAE: {:.2f}".format(round(gbm_mae, 2)))
print("GBM RMSE: {:.2f}".format(round(gbm_rmse, 2)))


# In[]:
    # here we'll use dhs_0_1_wg as the response vector for the classifiers
x = datasetv2[["DHS_prop_repeat", 
                    "DHS_prop_GC", "DHS_length", "n_SNV_Zhou_per_bp", 
                    "distanceToTSS", "zeta.human", "zeta.chimp", "PP_con", "PP_acc", 
                    "PhastCons",
                    "chromHMM_cat_longest", 
                    "annotation", "PhyloP_primates_score"]]
y = datasetv2["dhs_0_1_wg"]

# In[]:
    # Make column transformer
column_transform = []
column_transform = make_column_transformer(
    (ohe, ['chromHMM_cat_longest','annotation']))

#Apply column transformer to predictor variables
column_transform.fit(x)


# In[]:
# Splitting train and split data
# The test data set size is 20% of the total records. This test data will not 
# be used in model training and work as an independent test data.
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2, random_state=0)
# In[]:

lr_pipeline.fit(x_train, y_train)
lr_predictions = lr_pipeline.predict(x_test)
print("First 5 LR predictions: ", list(lr_predictions[:5]))

sv_pipeline.fit(x_train, y_train)
sv_predictions = sv_pipeline.predict(x_test)
print("First 5 SVM predictions: ", list(sv_predictions[:5]))

#rf_pipeline.fit(x_train, y_train)
#rf_predictions = rf_pipeline.predict(x_test)
#print("First 5 RF predictions: ", list(rf_predictions[:5]))
# To use random forest, need binary outcome

# In[]:
# With predictions ready from the two pipelines, we can proceed to evaluate the 
# accuracy of these predictions using mean absolute error (MAE) and mean squared 
# error (RMSE).
# Calculate mean square error and root mean squared error

lr_mae = mean_absolute_error(lr_predictions, y_test)
lr_rmse = np.sqrt(mean_squared_error(lr_predictions, y_test))
print("LR MAE: {:.2f}".format(round(lr_mae, 2)))
print("LR RMSE: {:.2f}".format(round(lr_rmse, 2)))

sv_mae = mean_absolute_error(sv_predictions, y_test)
sv_rmse = np.sqrt(mean_squared_error(sv_predictions, y_test))
print("SVM MAE: {:.2f}".format(round(sv_mae, 2)))
print("SVM RMSE: {:.2f}".format(round(sv_rmse, 2)))